# PSEi Stock Market

### Import Python Packages
This will import the dependencies to be used to download and analyze the data that will be downloaded through the Yahoo Finance API.

!pip install yfinance
!pip install numpy
!pip install pandas_datareader 
!pip install scikit-learn
!pip install plotly
!pip install quandl
!conda install -c conda-forge ta-lib

In [1]:
# Yahoo Finance API
import yfinance as yf  

# DataFrame
import pandas as pd

# Numerical Python
import numpy as np

# Pandas Data Reader
from pandas_datareader import data as pdr
from pandas_datareader import wb

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Technical Analysis Library
import talib as ta

# Plotly
import plotly.express as px
from plotly.subplots import make_subplots

### Library Options
This will set the options for the libraries that will be used in this notebook

In [2]:
yf.pdr_override()   # Override Yahoo Finance API output to use pandas data reader
pd.options.plotting.backend = "plotly" # Use plotly as the plotting backend

### Download PSEi Data
This will download the data from Yahoo Finance API and then import it to pandas dataframe.
The data will be downloaded is from `2000-01-01` to `2023-05-18` with an interval of `1 day`

In [3]:
psei_df = pdr.get_data_yahoo('PSEI.PS', '2000-01-01', '2023-05-18')
psei_df = psei_df.dropna()
psei_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,2143.669922,2148.709961,2122.989990,2141.770020,2141.219482,0
2000-01-04,2151.550049,2158.209961,2130.679932,2153.179932,2152.626465,0
2000-01-05,2113.379883,2113.379883,2070.139893,2074.750000,2074.216553,0
2000-01-06,2079.050049,2082.810059,2066.879883,2079.110107,2078.575684,0
2000-01-07,2079.320068,2094.290039,2077.649902,2094.290039,2093.751709,0


In [4]:
gdp = wb.download(indicator='NY.GDP.MKTP.CD', country='PH', start=2000, end=2023)
gdp = gdp.unstack().T.reset_index(0)
gdp.index = pd.to_datetime(gdp.index, format='%Y')

gdp_df = psei_df.join(gdp, how='outer')
gdp_df = gdp_df.interpolate(method='linear')
gdp_df = gdp_df.dropna()
gdp_df.rename(columns={"Philippines": "GDP (Current $)"}, inplace=True)
gdp_df.drop("level_0", axis=1, inplace=True)
gdp_df.to_csv('gdp_current.csv')
gdp_df

,Open,High,Low,Close,Adj Close,Volume,GDP (Current $)
2001-01-01,1486.600037,1491.395020,1459.770020,1471.494995,1471.116760,0.0,7.892123e+10
2002-01-01,1160.080017,1171.299988,1159.210022,1168.824951,1168.524475,0.0,8.430729e+10
2003-01-01,1016.955017,1018.850006,1009.279999,1013.524994,1013.264465,0.0,8.703915e+10
2004-01-01,1452.900024,1452.900024,1434.699951,1442.369995,1441.999146,0.0,9.500203e+10
2005-01-01,1825.500000,1839.525024,1818.914978,1834.864990,1834.393311,0.0,1.074200e+11
2006-01-01,2080.135010,2104.165039,2079.369995,2102.915039,2102.374390,0.0,1.276529e+11
2007-01-01,2966.160034,2984.334961,2956.369995,2979.695068,2978.928955,0.0,1.559804e+11
2008-01-01,3639.300049,3651.015015,3607.305054,3619.445068,3618.514648,0.0,1.816246e+11
2009-01-01,1892.145020,1928.149963,1887.769958,1923.769958,1923.275391,0.0,1.759747e+11
2010-01-01,3043.714966,3052.479980,3019.964966,3028.844971,3028.066406,0.0,2.083687e+11


In [5]:
inflation_rate = wb.download(indicator='FP.CPI.TOTL.ZG', country='PH', start=2000, end=2023)
inflation_rate = inflation_rate.unstack().T.reset_index(0)
inflation_rate.index = pd.to_datetime(inflation_rate.index, format='%Y')

inflation_rate_df = gdp_df.join(inflation_rate, how='outer')
inflation_rate_df = inflation_rate_df.interpolate(method='linear')
inflation_rate_df = inflation_rate_df.dropna()
inflation_rate_df.rename(columns={"Philippines": "Inflation"}, inplace=True)
inflation_rate_df.drop("level_0", axis=1, inplace=True)
inflation_rate_df.to_csv('inflation_rate.csv')
inflation_rate_df

,Open,High,Low,Close,Adj Close,Volume,GDP (Current $),Inflation
2001-01-01,1486.600037,1491.395020,1459.770020,1471.494995,1471.116760,0.0,7.892123e+10,5.345502
2002-01-01,1160.080017,1171.299988,1159.210022,1168.824951,1168.524475,0.0,8.430729e+10,2.722772
2003-01-01,1016.955017,1018.850006,1009.279999,1013.524994,1013.264465,0.0,8.703915e+10,2.289157
2004-01-01,1452.900024,1452.900024,1434.699951,1442.369995,1441.999146,0.0,9.500203e+10,4.829211
2005-01-01,1825.500000,1839.525024,1818.914978,1834.864990,1834.393311,0.0,1.074200e+11,6.516854
2006-01-01,2080.135010,2104.165039,2079.369995,2102.915039,2102.374390,0.0,1.276529e+11,5.485232
2007-01-01,2966.160034,2984.334961,2956.369995,2979.695068,2978.928955,0.0,1.559804e+11,2.900000
2008-01-01,3639.300049,3651.015015,3607.305054,3619.445068,3618.514648,0.0,1.816246e+11,8.260447
2009-01-01,1892.145020,1928.149963,1887.769958,1923.769958,1923.275391,0.0,1.759747e+11,4.219031
2010-01-01,3043.714966,3052.479980,3019.964966,3028.844971,3028.066406,0.0,2.083687e+11,3.789836


In [6]:
def plotly_dual_axis(data1,data2, title="", y1="", y2=""):
    # Create subplot with secondary axis
    subplot_fig = make_subplots(specs=[[{"secondary_y": True}]])

    #Put Dataframe in fig1 and fig2
    fig1 = px.line(data1)
    fig2 = px.line(data2)
    #Change the axis for fig2
    fig2.update_traces(yaxis="y2")

    #Add the figs to the subplot figure
    subplot_fig.add_traces(fig1.data + fig2.data)

    #FORMAT subplot figure
    subplot_fig.update_layout(title=title, yaxis=dict(title=y1), yaxis2=dict(title=y2))

    #RECOLOR so as not to have overlapping colors
    subplot_fig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))


    return subplot_fig

### Output the raw data to CSV

In [7]:
psei_df.to_csv('raw.csv')

# Initialize Variables

### Time Period
The window size (moving average in days) of the rolling mean and rolling correlation

### Train Size
The percentage of the data that will be used for training the model

In [8]:
# Time Peroid (in days)
timeperiod = 10

# Train Size (in percentage 0.1 = 10%)
trainsize = 0.8

# Prepare the data for the model

### Populate Tables

In [9]:
df = psei_df.copy()

df['S_' + str(timeperiod)] = df['Close'].rolling(window=timeperiod).mean() # Rolling mean

df['Corr'] = df['Close'].rolling(window=timeperiod).corr(df['S_' + str(timeperiod)]) # Correlation between the close price and the rolling mean

df['RSI'] = ta.RSI(np.array(df['Close']), timeperiod=timeperiod) # Relative Strength Index

df['Open-Close'] = df['Open'] - df['Close'].shift(1) # The difference between the current day's open and the previous day's close

df['Open-Open'] = df['Open'] - df['Open'].shift(1) # The difference between the current day's open and the previous day's open

df = df.dropna() # Drop the NaN values
df.head() #    Show the dataframe

,Open,High,Low,Close,Adj Close,Volume,S_10,Corr,RSI,Open-Close,Open-Open
Date,,,,,,,,,,,
2000-01-27,1993.709961,2011.099976,1970.780029,1975.199951,1974.692139,0,2051.536023,0.945510,23.487207,-4.880005,-53.570068
2000-01-28,1974.310059,1978.650024,1960.439941,1969.390015,1968.883789,0,2038.265015,0.952497,22.802548,-0.889893,-19.399902
2000-01-31,1959.000000,1997.089966,1950.010010,1989.430054,1988.918579,0,2027.485022,0.910715,30.560290,-10.390015,-15.310059
2000-02-01,1986.619995,1990.949951,1972.109985,1973.439941,1972.932617,0,2017.345007,0.901408,28.060317,-2.810059,27.619995
2000-02-02,1976.099976,1988.030029,1971.880005,1975.959961,1975.452026,0,2010.088000,0.890174,29.076283,2.660034,-10.520020


In [10]:
x = df.iloc[:,:9] # The features that will be used for the model
y = np.where(df['Close'].shift(-1) > df['Close'],1,-1) # 1 if the price goes up, -1 otherwise

In [11]:
split = int(trainsize * len(df)) # Split the data into train and test set
x_train, x_test, y_train, y_test = x[:split], x[split:], y[:split], y[split:] # Split the data into train and test set

## Implement Logistic Regression

In [12]:
logistic_regression_model = LogisticRegression()
logistic_regression_model = logistic_regression_model.fit(x_train, y_train)

## Implement Linear Regression

In [13]:
from sklearn.linear_model import LinearRegression
import datetime
import quandl

quan_psei_df = pd.DataFrame(psei_df, columns=['Close'])
quan_psei_df.reset_index(inplace=True)
data_train, data_test = train_test_split(quan_psei_df, test_size = 0.2)

X_train = np.array(data_train.index).reshape(-1, 1)
y_train = data_train['Close']

linear_regression_model = LinearRegression()  # Initialize the model
linear_regression_model = linear_regression_model.fit(X_train, y_train)  # Fit the model

In [14]:
data_train['Predict'] = linear_regression_model.predict(X_train)

In [15]:
X_train.size
data_train = data_train.sort_values(by='Date')

### Show the model

In [16]:
# Show the coefficients of the model
pd.DataFrame(zip(x.columns, np.transpose(logistic_regression_model.coef_)))
# 1st column is the features, 2nd column is the coefficients

,0,1
0,Open,[2.2167891450125878e-06]
1,High,[2.241330953471385e-06]
2,Low,[2.2182630397449996e-06]
3,Close,[2.237755014261298e-06]
4,Adj Close,[2.2371797880817864e-06]
5,Volume,[-3.352507710482293e-09]
6,S_10,[2.253239183341203e-06]
7,Corr,[7.71933511806674e-11]
8,RSI,[1.3118414601767827e-08]


### Predict the price

In [17]:
# Predict X
probability = logistic_regression_model.predict_proba(x_test)

probability_df = pd.DataFrame(probability)
probability_df


# 1st column is the probability of the price going down
# 2nd column is the probability of the price going up

,0,1
0,0.474604,0.525396
1,0.474267,0.525733
2,0.474196,0.525804
3,0.474031,0.525969
4,0.474062,0.525938
...,...,...
1152,0.477758,0.522242
1153,0.477964,0.522036
1154,0.478091,0.521909
1155,0.478010,0.521990


In [18]:
# Predict Y
y_predicted = logistic_regression_model.predict(x_test) # Predict the price

### Confusion Matrix

In [19]:
y_confusion_matrix = metrics.confusion_matrix(y_test, y_predicted)

y_confusion_matrix_df = pd.DataFrame(y_confusion_matrix).transpose()
y_confusion_matrix_df # Show the confusion matrix

,0,1
0,1,0
1,563,593


### Show the accuracy of the model

In [20]:
print("logistic_regression_model Accuracy:", logistic_regression_model.score(x_test,y_test)) # Show the accuracy of the model

X_reg_test = np.array(data_test.index).reshape(-1, 1)
y_reg_test = data_test['Close']
print("linear_regression_model Accuracy:", linear_regression_model.score(X_reg_test,y_reg_test)) # Show the accuracy of the model



logistic_regression_model Accuracy: 0.5133967156439067
linear_regression_model Accuracy: 0.8286892638066767


Logistic Regression model Accuracy: 51%


Linear Regression model Accuracy: 83%

In [21]:
report = metrics.classification_report(y_test, y_predicted, output_dict=True) # Show the classification report
report_df = pd.DataFrame(report).transpose()
report_df


,precision,recall,f1-score,support
-1,1.000000,0.001773,0.003540,564.000000
1,0.512976,1.000000,0.678102,593.000000
accuracy,0.513397,0.513397,0.513397,0.513397
macro avg,0.756488,0.500887,0.340821,1157.000000
weighted avg,0.750384,0.513397,0.349275,1157.000000


### Show the cross validation score

In [22]:
cross_val = cross_val_score(LogisticRegression(), x, y, scoring='accuracy', cv=10)
cross_val


array([0.50604491, 0.50431779, 0.47841105, 0.50604491, 0.50519031,
       0.50692042, 0.50692042, 0.50865052, 0.50865052, 0.51557093])

### Predict Future

### Calculate Prediction Signals

In [23]:
df['Predicted_Signal'] = logistic_regression_model.predict(x) # Predict the signal
df['PSEi_returns'] = np.log(df['Close'] / df['Close'].shift(1)) # Calculate the PSEi returns
Cumulative_PSEi_returns = np.cumsum(df[split:]['PSEi_returns']) # Calculate the cumulative PSEi returns

df['Strategy_Returns'] = df['PSEi_returns'] * df['Predicted_Signal'].shift(1) # Calculate the strategy returns
Cumulative_Strategy_returns = np.cumsum(df[split:]['Strategy_Returns']) # Calculate the cumulative strategy returns

### Output the data to CSV

In [24]:
df.to_csv('predicted signals.csv') # Output the data to CSV

# Plot the data

In [ ]:
### Plot PSEi Close

In [25]:
quan_psei_df.plot(x='Date',y=['Close'])

In [ ]:
### Plot Future Prediction

In [26]:
some_row = pd.DataFrame(index=pd.date_range('2023-05-23', periods=365, freq='D'), columns=['Close', 'Predict'])
some_row.index.name = 'Date'
some_row.reset_index(inplace=True)
#some_row = some_row.fillna()

#some_row
prediction_row = pd.concat([data_train, some_row])
prediction_row = prediction_row.sort_values(by='Date')
prediction_row = prediction_row.reset_index(drop=True)

prediction_row[:4641]['Predict'] = linear_regression_model.predict(X_train)
#prediction_row[4641:]["Predict"] = 


for idx, row in prediction_row.iterrows():
    prediction_row.loc[idx,'Predict'] = float(linear_regression_model.predict([[idx]]))

#prediction_row
prediction_row.plot(x='Date', y=['Predict', 'Close'])

/tmp/ipykernel_7107/1944058992.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
prediction_row.to_csv('linear_prediction.csv')

### Plot the close price and the rolling mean

In [28]:
df.plot(y=['Close', 'S_' + str(timeperiod)], labels={'value':'Value', 'index':'Date', 'variable':'Variables'}) # Plot the close price and the rolling mean

### Plot Cumulative PSEi returns

In [29]:
Cumulative_PSEi_returns.plot(labels={'value':'Cumulative Returns', 'index':'Date', 'variable':'Variables'}) # Plot the cumulative PSEi returns

### Plot the Cumulative Strategy Returns

In [30]:
Cumulative_Strategy_returns.plot(labels={'value':'Cumulative Returns', 'index':'Date', 'variable':'Variables'}) # Plot the cumulative strategy returns

### Plot the PSEi returns and the Strategy returns

In [31]:
Cumulative_PSEi_returns_df = Cumulative_PSEi_returns.to_frame()
Cumulative_Strategy_returns_df = Cumulative_Strategy_returns.to_frame()

cumulative_returns_df = pd.merge(Cumulative_PSEi_returns_df, Cumulative_Strategy_returns_df, left_index=True, right_index=True)
cumulative_returns_df.head()

,PSEi_returns,Strategy_Returns
Date,,
2018-08-22,0.017410,0.017410
2018-08-23,0.039667,0.039667
2018-08-24,0.034842,0.034842
2018-08-28,0.044853,0.044853
2018-08-29,0.043111,0.043111


In [32]:
cumulative_returns_df.plot(title='PSEi and Strategy Cumulative Returns', labels={'value':'Cumulative Returns', 'index':'Date', 'variable':'Strategy'})

In [33]:
cumulative_returns_df.to_csv('cumulative_returns.csv') # Output the data to CSV

### Plot GDP vs PSEI

In [34]:
plotly_dual_axis(df['Adj Close'], gdp_df['GDP (Current $)'], title="PSEi vs GDP", y1="PSEi", y2="GDP")

### Plot Inflation Rate

In [35]:
inflation_rate_df.plot(y=["Inflation"], title="Inflation Rate (annual %)")

### Plot PSEi Open-Close

In [36]:
df.plot(y=['Open', 'Close'])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=104f152a-ece5-47f4-a4d4-d9ea9c0d19d6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>